In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import dlt

## **Building Lakeflow pipeline**

In [0]:
@dlt.view
def transform_flights():
  df = spark.readStream.format("delta")\
                  .load("/Volumes/workspace_bryanq/bronze/bronze_volume/flights/data/")\
                  .withColumn("flight_date", to_date(col("flight_date")))\
                  .drop("_rescued_data")
  return df

In [0]:
dlt.create_streaming_table("silver_flights")

dlt.create_auto_cdc_flow(
  target = "silver_flights",
  source = "transform_flights",
  keys = ["flight_id"],
  sequence_by = col("flight_id"),
  stored_as_scd_type = 1
)